In [5]:
from policy import PIB, NEATProperty, NEATPolicy, NEATRequest
from cib import CIB

In [6]:
cib = CIB('cib/example/')
pib = PIB()

loading CIB source A.connection
loading CIB source B.connection
loading CIB source C.connection


We define to policies and add them to the PIB

In [7]:
policy1 = NEATPolicy(name='Bulk transfer')
policy1.match.insert(NEATProperty(('remote_ip', '10.1.23.45')))
policy1.properties.insert(NEATProperty(('capacity', (10, 100)), level=NEATProperty.IMMUTABLE))
policy1.properties.insert(NEATProperty(('MTU', 9600)))

policy2 = NEATPolicy(name='TCP options')
policy2.match.insert(NEATProperty(('MTU', 9600)))
policy2.match.insert(NEATProperty(('is_wired', True)))
policy2.properties.insert(NEATProperty(('TCP_window_scale', True)))

pib.register(policy2)
pib.register(policy1)
pib.dump()

===== PIB START =====
POLICY Bulk transfer: {(remote_ip|10.1.23.45)}  ==>  {(MTU|9600), [capacity|10-100]}
POLICY TCP options: {(MTU|9600), (is_wired|True)}  ==>  {(TCP_window_scale|True)}
===== PIB END =====
